# MIMIV-IV Preprocessing Code
The preprocessing is based on MIMIC-Extract (https://arxiv.org/pdf/1907.08322.pdf). <br>
We use MIMIV-IV-1.0 as the (https://physionet.org/content/mimiciv/1.0/). <br>

Steps: <br>
(1) Download MIMIC-IV-1.0 from physionet (link above). <br>
(2) Run the notebook from the first cell to the last. This will take about 15 hrs to complete running. <br>
(3) To use this with our EMDOT python package, run MIMICicuDatasetProcessor.py with "MIMIC_icu_cate_raw.csv" as the input file. <br>

<br>

Notice: As some of the processes take a while, we manually save the intermediate results (as dataframes). The dataframe outputs at the end of this notebook (i.e. MIMIC_icu_cate_raw.csv) is the fully-processed MIMIC-IV data.

## Import libraries

In [ ]:
import os
import yaml
import json

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
# To test this code on a subset of patients, set N to a small number
N = None
# N = 10

## 1. Load icu_stay table & Select distinct patient based on first admitted time

In [ ]:
## The parent directory for MIMIC-IV dataset
parent_dir = ''  #TODO: Enter the path to folder containing MIMIC-IV dataset

data_dir = os.path.join(parent_dir, "physionet.org", "files", "mimiciv", "1.0")

In [ ]:
df_icu_stay = pd.read_csv(data_dir + "/icu/icustays.csv.gz", compression='gzip', header=0, index_col=None)

In [ ]:
print(df_icu_stay.shape)
df_icu_stay.head()

In [ ]:
df_icu_stay["intime"] = pd.to_datetime(df_icu_stay["intime"])
df_icu_stay["outtime"] = pd.to_datetime(df_icu_stay["outtime"])
df_icu_stay = df_icu_stay.loc[df_icu_stay.groupby("subject_id")["intime"].idxmin()]
df_icu_stay.shape

In [ ]:
if N is not None:
    df_icu_stay = df_icu_stay.iloc[:N, :]

### Checking if there is only one record for each id
Expect no printing!

In [ ]:
for sub_id, df_by_id in df_icu_stay.groupby("stay_id"):
    if df_by_id.shape[0] > 1:
        print(sub_id)

## 2. Load core_patient table & Merge by subject_id

In [ ]:
df_core_patient = pd.read_csv(data_dir + "/core/patients.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
print(df_core_patient.shape)
df_core_patient.head()

In [ ]:
df_icu_stay_merge = df_icu_stay.merge(df_core_patient, left_on="subject_id", right_on="subject_id", how="inner")

In [ ]:
print(df_icu_stay_merge.shape)
df_icu_stay_merge.head()

## 3. Load chart_events table & d_items table

In [ ]:
df_chart_event = pd.read_csv(data_dir + "/icu/chartevents.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
df_chart_event["charttime"] = pd.to_datetime(df_chart_event["charttime"])

In [ ]:
print(df_chart_event.shape)
df_chart_event.head()

In [ ]:
df_items = pd.read_csv(data_dir + "/icu/d_items.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
print(df_items.shape)
df_items.head()

In [ ]:
df_items[df_items["linksto"] == "procedureevents"]

### Select the latest value of each event in 24 hours

In [ ]:
df_items_chart = df_items[df_items["linksto"] == "chartevents"]

In [ ]:
print(df_items_chart.shape)
len(df_items_chart["itemid"].unique().tolist())

### Add new empty columns of corresponding chartevent

In [ ]:
df_icu_stay_merge[df_items_chart["abbreviation"].unique().tolist()] = np.nan

In [ ]:
print(df_icu_stay_merge.shape)
df_icu_stay_merge.head()

In [ ]:
for sub_id, df_by_sub_id in tqdm(df_icu_stay_merge.groupby("subject_id"), desc="sub_id", leave=False):
    df_chart_tmp = df_chart_event[df_chart_event["subject_id"] == sub_id]
    ## Select the measurement that is happened after 24 hours of the admit time
    df_chart_tmp = df_chart_tmp[(df_chart_tmp["charttime"] - df_by_sub_id["intime"].item()) <= pd.Timedelta(days=1)]
    
    for item_id, df_by_item_id in tqdm(df_chart_tmp.groupby("itemid"), desc="item_id", leave=False):
        df_by_item_id = df_by_item_id.loc[df_by_item_id["charttime"].idxmax()]
        df_icu_stay_merge.loc[df_icu_stay_merge["subject_id"] == sub_id, df_items_chart[df_items_chart["itemid"] == item_id]["abbreviation"].item()] = df_by_item_id["value"]
        df_by_sub_id[df_items_chart[df_items_chart["itemid"] == item_id]["abbreviation"]] = df_by_item_id["value"]
    

In [ ]:
print(df_icu_stay_merge.shape)
df_icu_stay_merge.head()

In [ ]:
df_icu_stay_merge.to_csv("MIMIC_IV_icu_chart.csv")

### drop constant

In [ ]:
df_icu_stay_merge.dropna(axis=1, how="all")

In [ ]:
df_icu_stay_merge.to_csv("MIMIC_IV_icu_chart_drop_all_nan.csv")

In [ ]:
df_icu_stay_merge = pd.read_csv("MIMIC_IV_icu_chart_drop_all_nan.csv", index_col=0, low_memory=False)

In [ ]:
print(df_icu_stay_merge.shape)
df_icu_stay_merge.head()

In [ ]:
df_icu_stay_merge["intime"] = pd.to_datetime(df_icu_stay_merge["intime"])

In [ ]:
middle_year = df_icu_stay_merge["anchor_year_group"].values
middle_year = [int(f[:4]) + 1 for f in middle_year]
df_icu_stay_merge["admit_year"] = middle_year + df_icu_stay_merge["intime"].dt.year - df_icu_stay_merge["anchor_year"]

In [ ]:
np.sort(df_icu_stay_merge["admit_year"].unique())

In [ ]:
for year, df_by_year in df_icu_stay_merge.groupby("admit_year"):
    num = df_by_year[df_by_year["los"] > 3].shape[0]
    print(f"Year: {year}   # sample: {df_by_year.shape[0]}   # los > 3 sample: {num}")

## 4. Load procedure table & d_items table

In [ ]:
df_procedure_event = pd.read_csv(data_dir + "/icu/procedureevents.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
df_procedure_event["starttime"] = pd.to_datetime(df_procedure_event["starttime"])
df_procedure_event["endtime"] = pd.to_datetime(df_procedure_event["endtime"])

In [ ]:
print(df_procedure_event.shape)
df_procedure_event.head()

In [ ]:
df_items = pd.read_csv(data_dir + "/icu/d_items.csv.gz", compression='gzip', header=0, index_col = None)

### Select the latest value of each event in 24 hours

In [ ]:
df_items_procedure = df_items[df_items["linksto"] == "procedureevents"]

In [ ]:
print(df_items_procedure.shape)
df_items_procedure.head()

In [ ]:
df_icu_stay_merge[df_items_procedure["abbreviation"].unique().tolist()] = np.nan

In [ ]:
for sub_id, df_by_sub_id in tqdm(df_icu_stay_merge.groupby("subject_id"), desc="sub_id", leave=False):
    df_procedure_tmp = df_procedure_event[df_procedure_event["subject_id"] == sub_id]
    ## Select the measurement that is happened after 24 hours of the admit time
    df_procedure_tmp = df_procedure_tmp[(df_procedure_tmp["endtime"] - df_by_sub_id["intime"].item()) <= pd.Timedelta(days=1)]
    
    for item_id, df_by_item_id in tqdm(df_procedure_tmp.groupby("itemid"), desc="item_id", leave=False):
        df_by_item_id = df_by_item_id.loc[df_by_item_id["endtime"].idxmax()]
        df_icu_stay_merge.loc[df_icu_stay_merge["subject_id"] == sub_id, df_items_procedure[df_items_procedure["itemid"] == item_id]["abbreviation"].item()] = df_by_item_id["value"]

In [ ]:
print(df_icu_stay_merge.shape)
df_icu_stay_merge.head()

In [ ]:
df_icu_stay_merge.to_csv("MIMIC_IV_icu_chart_procedure.csv")

### drop constant

In [ ]:
df_icu_stay_merge.dropna(axis=1, how="all", inplace=True)

In [ ]:
df_icu_stay_merge.to_csv("MIMIC_IV_icu_chart_drop_all_nan.csv")

In [ ]:
df_icu_stay_merge = pd.read_csv("MIMIC_IV_icu_chart_drop_all_nan.csv", index_col=0, low_memory=False)

In [ ]:
df_icu_stay_merge.head()

In [ ]:
df_icu_stay_merge["intime"] = pd.to_datetime(df_icu_stay_merge["intime"])

## 5. Load lab table & d_items table

In [ ]:
df_lab_event = pd.read_csv(data_dir + "/hosp/labevents.csv.gz", compression='gzip', header=0, index_col=None)

In [ ]:
print(df_lab_event.shape)
df_lab_event.head()

In [ ]:
df_lab_event["charttime"] = pd.to_datetime(df_lab_event["charttime"])

In [ ]:
df_items = pd.read_csv(data_dir + "/hosp/d_labitems.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
df_items.loc[df_items["label"] == " ", "label"] = df_items.loc[df_items["label"] == " ", "fluid"]

In [ ]:
print(df_items.shape)
df_items.head()

### Select the latest value of each event in 24 hours

In [ ]:
df_icu_stay_merge[df_items["label"].unique().tolist()] = np.nan

In [ ]:
for sub_id, df_by_sub_id in tqdm(df_icu_stay_merge.groupby("subject_id"), desc="sub_id", leave=False, mininterval=0):
    df_lab_tmp = df_lab_event[df_lab_event["subject_id"] == sub_id]
    ## Select the measurement that is happened after 24 hours of the admit time
    df_lab_tmp = df_lab_tmp[(df_lab_tmp["charttime"] - df_by_sub_id["intime"].item()) <= pd.Timedelta(days=1)]
    
    for item_id, df_by_item_id in tqdm(df_lab_tmp.groupby("itemid"), desc="item_id", leave=False, mininterval=0):
        df_by_item_id = df_by_item_id.loc[df_by_item_id["charttime"].idxmax()]
        df_icu_stay_merge.loc[df_icu_stay_merge["subject_id"] == sub_id, df_items[df_items["itemid"] == item_id]["label"].item()] = df_by_item_id["value"]

In [ ]:
df_icu_stay_merge.dropna(axis=1, how="all", inplace=True)

In [ ]:
print(df_icu_stay_merge.shape)
df_icu_stay_merge.head()

## 6. Additional pre-processing 
(This section is pre-processed based on the above steps. If e.g. the thresholds are different, some columns pre-processed explicitly below may not exist.)

### Drop feature if the values are missed for more than half of the patients

In [ ]:
nan_portion_list = []

for col in tqdm(df_icu_stay_merge.columns.tolist(), desc="Col", leave=False):
    num = df_icu_stay_merge[df_icu_stay_merge[col].isnull()].shape[0] / df_icu_stay_merge.shape[0]
    nan_portion_list.append(num)

In [ ]:
col_list_index = [index for index, value in enumerate(nan_portion_list) if value <= 0.5]

In [ ]:
select_col = []

for index in col_list_index:
    select_col.append(df_icu_stay_merge.columns.tolist()[index])

In [ ]:
df_icu_stay_merge_selected = df_icu_stay_merge.loc[:, select_col]
df_icu_stay_merge_selected.to_csv("df_icu_selected.csv")

### Determine numerical and categorical features

In [ ]:
df_chart_procedure_id = pd.read_csv(data_dir + "/icu/d_items.csv.gz", compression='gzip', header=0, index_col = None)
df_lab_id = pd.read_csv(data_dir + "/hosp/d_labitems.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
df_lab_event = pd.read_csv(data_dir + "/hosp/labevents.csv.gz", compression='gzip', header=0, index_col=None)

In [ ]:
feature_dict_cate = {
    "Numerical": [],
    "Categorical": [],
    "label": []
}

NMETA = 12

feature_col = df_icu_stay_merge_selected.columns[NMETA:]

for col in feature_col:
    if col in df_chart_procedure_id["abbreviation"].tolist():
        if df_chart_procedure_id.loc[df_chart_procedure_id["abbreviation"] == col, "param_type"].iloc[0] == "Numberic":
            feature_dict_cate["Numerical"].append(col)
        else:
            feature_dict_cate["Categorical"].append(col)
    elif col in df_lab_id["label"].tolist():
        item_id_list = df_lab_id.loc[df_lab_id["label"] == col, "itemid"].tolist()
        for item_id in item_id_list:
            df_tmp = df_lab_event[df_lab_event["itemid"] == item_id]
            if df_tmp.shape[0] > 0:
                if df_tmp["valuenum"].isnull().iloc[0]:
                    feature_dict_cate["Categorical"].append(col)
                else:
                    feature_dict_cate["Numerical"].append(col)
                break
    else:
        print(col)

### Calculate patients' age

In [ ]:
df_icu_stay_merge_selected["intime"] = pd.to_datetime(df_icu_stay_merge_selected["intime"])
df_icu_stay_merge_selected["admit_age"] = df_icu_stay_merge_selected["intime"].dt.year - df_icu_stay_merge_selected["anchor_year"] + df_icu_stay_merge_selected["anchor_age"]

### Manually add features

In [ ]:
feature_dict_cate["Numerical"].append("admit_year")

In [ ]:
feature_dict_cate["Numerical"].append("admit_age")

In [ ]:
feature_dict_cate["Categorical"].append("gender")

In [ ]:
feature_dict_cate["label"].append("los")

In [ ]:
feature_dict_cate["label"].append("dod")

In [ ]:
df_icu_stay_merge_selected.to_csv("MIMIC_icu_cate_raw.csv")

### Save the feature information

In [ ]:
with open("feature_yaml_cate.yaml", "w+") as f:
    yaml.dump(feature_dict_cate, f)

In [ ]:
with open("feature_dict_cate.json", "w+") as f:
    json.dump(feature_dict_cate, f)

In [ ]:
for year, df_by_year in df_icu_stay_merge_selected.groupby("admit_year"):
    num = df_by_year[df_by_year["los"] > 3].shape[0]
    print(f"Year: {year}   # sample: {df_by_year.shape[0]}   # los > 3 sample: {num}")

### Clean up the dataset (Replace string with approximate numerical values)

In [ ]:
df_icu_stay_merge_selected["Epithelial Cells"].replace({
    '0-2': "1",
    '<1': "1",
    '6-10': "8",
    '11-20': "15.5",
    '21-50': "35",
    ' ': np.nan,
    '11/20': "15.5",
    'N': np.nan,
    '>50': "50",
    ' 0-2': "1",
    '3-5': "4"
}, inplace=True)

In [ ]:
df_icu_stay_merge_selected["INR(PT)"].replace({
    'UNABLE TO REPORT': np.nan,
    '>13.1': "13.1",
    'ERROR': np.nan,
    '>16.0': "16.0",
    'LAB ERROR': np.nan
}, inplace=True)

In [ ]:
df_icu_stay_merge_selected["pH"].replace({
    'NotDone': np.nan,
    ' ': np.nan
}, inplace=True)

In [ ]:
df_icu_stay_merge_selected["RBC"].replace({
    '3-5': "4",
    '6-10': "8",
    '0-2': "1",
    '>50': "50",
    '11-20': "15.5",
    '>1000': "1000",
    '21-50': "35",
    ' ': np.nan,
    '<1': "1",
    '0-20-2': "20",
    'O': "0"
}, inplace=True)

In [ ]:
df_icu_stay_merge_selected["Specific Gravity"].replace({
    '>1.035': "1.035",
    '>1.050': "1.050",
    '<1.005': "1.005",
    '<=1.005': "1.005",
    '>1.040': "1.040",
    '>=1.035': "1.035",
    ' ': np.nan,
    '>1.030': "1.030",
    '> 1.035': "1.035",
    '>1.038': "1.038",
    '>=1.030': "1.030",
    '1..025': "1.025",
    '.1.012': "1.012"
}, inplace=True)

### Add feature "dod"

In [ ]:
df_core_patient = pd.read_csv(data_dir + "/core/patients.csv.gz", compression='gzip', header=0, index_col = None)

In [ ]:
df_core_patient = df_core_patient.drop(columns=["gender", "anchor_age", "anchor_year", "anchor_year_group"])

In [ ]:
df_icu_stay_merge_selected = df_icu_stay_merge_selected.merge(df_core_patient, left_on="subject_id", right_on="subject_id", how="inner")

# Save processed data

In [ ]:
## This is fully-processed MIMIC-IV data
df_icu_stay_merge_selected.rename(columns={"Activity ": "Activity"}).to_csv("MIMIC_icu_cate_raw.csv")